In [1]:
# ==============================================================================
# CÉLULA 1: SETUP E INICIALIZAÇÃO DA ANÁLISE
# ==============================================================================
# Descrição: Importa bibliotecas, configura o ambiente e inicializa o
# AnalisadorBancario, que será a ferramenta para todas as coletas de dados.
# ------------------------------------------------------------------------------
import pandas as pd
from pathlib import Path
import sys
import numpy as np

print("--- 1. INICIANDO SETUP ---")

# Adiciona o diretório 'Code' ao path para encontrar o DataUtils
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

from DataUtils import AnalisadorBancario

# Configurações de exibição do Pandas
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100)

# Inicializa o Analisador (carrega todos os dados em memória)
output_dir = Path('..').resolve() / 'Output'
analisador = AnalisadorBancario(diretorio_output=str(output_dir))

# Garante que a pasta de resultados exista
results_dir = Path('.') / 'Results'
results_dir.mkdir(parents=True, exist_ok=True)

print("--- SETUP CONCLUÍDO. ANALISADOR PRONTO. ---")
print("-" * 60)

--- 1. INICIANDO SETUP ---
Iniciando o Analisador Bancário...
Analisador Bancário iniciado com sucesso!
  - 677,413 linhas em COSIF Individual
  - 567,956 linhas em COSIF Prudencial
  - 8,528,071 linhas em IFDATA Valores
  - 190,846 linhas em IFDATA Cadastro
  - Mapeamento interno criado para 6,365 nomes únicos.
--- SETUP CONCLUÍDO. ANALISADOR PRONTO. ---
------------------------------------------------------------


In [ ]:
# ==============================================================================
# CÉLULA 2: PAINEL DE CONTROLE DA ANÁLISE
# ==============================================================================
# Descrição: Este é o único lugar que você precisa alterar para configurar
# todo o relatório. Defina aqui os bancos, indicadores e o período desejado.
# ------------------------------------------------------------------------------
print("--- 2. CONFIGURANDO PARÂMETROS DA ANÁLISE ---")

# --- Período da Análise ---
DATA_INICIO_SERIES = 202201
DATA_FIM_SERIES = 202412
DATA_SNAPSHOT = 202412

# --- Geração Inteligente de Datas ---
data_inicio_dt = pd.to_datetime(f'{DATA_INICIO_SERIES}', format='%Y%m')
data_fim_dt = pd.to_datetime(f'{DATA_FIM_SERIES}', format='%Y%m') + pd.tseries.offsets.MonthEnd(0)

datas_mensais = pd.date_range(
    start=data_inicio_dt,
    end=data_fim_dt,
    freq='ME'
).strftime('%Y%m').astype(int).tolist()

datas_trimestrais = pd.date_range(
    start=data_inicio_dt,
    end=data_fim_dt,
    freq='QE'
).strftime('%Y%m').astype(int).tolist()

# --- Lista de Bancos ---
try:
    caminho_csv = Path('.') / 'Banks.csv'
    df_bancos_interesse = pd.read_csv(caminho_csv, dtype={'CNPJ_8': str}, sep=';')
    LISTA_BANCOS = [b for b in df_bancos_interesse['CNPJ_8'].tolist() if b]
except FileNotFoundError:
    print(f"ERRO CRÍTICO: O arquivo '{caminho_csv.name}' não foi encontrado. A análise será interrompida.")
    LISTA_BANCOS = []

# --- GRUPO 1: INDICADORES PARA SÉRIES TEMPORAIS (HISTÓRICO COMPLETO) ---
INDICADORES_SERIES_TEMPORAIS = {
    'Ativo Total': {'fonte': 'IFDATA'},
    'RWA': {'fonte': 'IFDATA', 'conta': 'Patrimônio de Referência para Comparação com o RWA'},
    'Índice de Basileia': {'fonte': 'IFDATA'},
    'Lucro Líquido': {'fonte': 'IFDATA'},
    'Patrimônio Líquido': {'fonte': 'IFDATA'},
    'Volume de Operações tipo G': {'fonte': 'IFDATA', 'conta': 'G'},
    'Volume de Operações tipo H': {'fonte': 'IFDATA', 'conta': 'H'},
    'Total de Volume de Operações': {'fonte': 'IFDATA', 'conta': 'Total Geral'}
}

# --- GRUPO 2: INDICADORES PARA SNAPSHOT (APENAS DATA MAIS RECENTE) ---
INDICADORES_SNAPSHOT = {
        'Volume de Operações tipo A': {'fonte': 'IFDATA', 'conta': 'A'},
        'Volume de Operações tipo AA': {'fonte': 'IFDATA', 'conta': 'AA'},
        'Volume de Operações tipo B': {'fonte': 'IFDATA', 'conta': 'B'},
        'Volume de Operações tipo C': {'fonte': 'IFDATA', 'conta': 'C'},
        'Volume de Operações tipo D': {'fonte': 'IFDATA', 'conta': 'D'},
        'Volume de Operações tipo E': {'fonte': 'IFDATA', 'conta': 'E'},
        'Volume de Operações tipo F': {'fonte': 'IFDATA', 'conta': 'F'},
        'Depósitos a Prazo': {'fonte': 'IFDATA', 'conta': 'Depósitos a Prazo (a4)'},
        'Letra de Crédito Imobiliário': {'fonte': 'IFDATA', 'conta': 'Letras de Crédito Imobiliário (c1)'},
        'Letra de Crédito Agronegócio': {'fonte': 'IFDATA', 'conta': 'Letras de Crédito do Agronegócio (c2)'},
        'Letras Financeiras': {'fonte': 'IFDATA', 'conta': 'Letras Financeiras (c3)'}
        }

# --- ATRIBUTOS CADASTRAIS (DADOS ESTÁTICOS) ---
ATRIBUTOS_CADASTRAIS = ['TCB_IFD_CAD']

print("--- CONFIGURAÇÃO CARREGADA. ---")
print("-" * 60)

--- 2. CONFIGURANDO PARÂMETROS DA ANÁLISE ---
--- CONFIGURAÇÃO CARREGADA. ---
------------------------------------------------------------


In [3]:
# ==============================================================================
# CÉLULA 3: COLETA OTIMIZADA E EXPORTAÇÃO DIRETA (VERSÃO FINAL)
# ==============================================================================
# Descrição: Este pipeline usa a técnica de pré-filtragem para máxima performance
# e exporta os componentes brutos diretamente para o Power BI, onde os
# cálculos de NPL e ROE serão feitos como Medidas DAX.
# ------------------------------------------------------------------------------
if LISTA_BANCOS:
    # --- ETAPA 1: Pré-filtragem para Performance ("Bulk Filtering") ---
    print("--- 3.1. Otimização: Pré-filtrando DataFrames para máxima performance... ---")
    
    todos_indicadores = {**INDICADORES_SERIES_TEMPORAIS, **INDICADORES_SNAPSHOT}
    contas_para_buscar = [info.get('conta', nome) for nome, info in todos_indicadores.items()]

    # Filtra os DataFrames base UMA ÚNICA VEZ
    # Inclui busca por nome e por código para máxima compatibilidade
    df_ifdata_trabalho = analisador.df_ifd_val[
        (analisador.df_ifd_val['NOME_CONTA_IFD_VAL'].isin(contas_para_buscar)) | 
        (analisador.df_ifd_val['CONTA_IFD_VAL'].astype(str).isin(contas_para_buscar))
    ].copy()

    # Guarda os DataFrames originais para restaurar depois
    original_ifd_val = analisador.df_ifd_val
    
    # "Injeta" o DataFrame otimizado no analisador para esta sessão
    analisador.df_ifd_val = df_ifdata_trabalho
    print(f" -> DataFrame IF.DATA pré-filtrado de {len(original_ifd_val):,} para {len(df_ifdata_trabalho):,} linhas.")
    
    # --- ETAPA 2: Coleta de Dados (agora usando os DFs otimizados) ---
    dfs_coletados = []
    print("\n--- 3.2. Coletando todos os componentes necessários... ---")
    
    for banco in LISTA_BANCOS:
        for nome_indicador, info in todos_indicadores.items():
            # Define a lista de datas correta para cada indicador
            if nome_indicador in INDICADORES_SERIES_TEMPORAIS:
                datas_para_buscar = datas_trimestrais if info['fonte'].upper() == 'IFDATA' else datas_mensais
            else:
                datas_para_buscar = [DATA_SNAPSHOT]
            
            conta_busca = info.get('conta', nome_indicador)
            
            df_serie = analisador.get_serie_temporal_indicador(
                identificador=banco, conta=conta_busca,
                datas=datas_para_buscar, fonte=info['fonte'], documento_cosif=info.get('documento'),
                fillna=0
            )
            if not df_serie.empty:
                dfs_coletados.append(df_serie)

    # Restaura o DataFrame original no analisador para não afetar outras células
    analisador.df_ifd_val = original_ifd_val
    print("\n -> Coleta concluída. DataFrame original restaurado.")

    if not dfs_coletados:
        print("\nNenhum dado foi encontrado para os parâmetros definidos.")
    else:
        # --- ETAPA 3: Consolidação e Exportação Final ---
        print("\n--- 3.3. Consolidando e exportando dados para o Power BI... ---")
        df_long_base = pd.concat(dfs_coletados, ignore_index=True)
        
        # Traduz os nomes técnicos para os nomes amigáveis
        mapa_nomes = {info.get('conta', nome): nome for nome, info in todos_indicadores.items()}
        df_long_base['Indicador'] = df_long_base['Conta'].replace(mapa_nomes)
        
        # Junção dos atributos cadastrais
        df_atributos = analisador.get_atributos_cadastro(identificador=LISTA_BANCOS, atributos=ATRIBUTOS_CADASTRAIS)
        df_para_bi = pd.merge(df_long_base, df_atributos, on=['CNPJ_8', 'Nome_Entidade'], how='left')

        # Seleciona e reordena as colunas finais
        colunas_finais = ['DATA', 'Nome_Entidade', 'CNPJ_8', 'Indicador', 'Valor'] + [col for col in ATRIBUTOS_CADASTRAIS if col in df_para_bi.columns]
        df_para_bi = df_para_bi[colunas_finais]

        nome_arquivo_saida = f'relatorio_powerbi_202412.xlsx'
        caminho_saida = results_dir / nome_arquivo_saida
        df_para_bi.to_excel(caminho_saida, index=False, engine='openpyxl')

        print("\n" + "="*60)
        print(">>> SUCESSO! <<<")
        print(f"Relatório de COMPONENTES para o Power BI foi gerado com {len(df_para_bi):,} linhas.")
        print(f"Arquivo salvo em: '{caminho_saida.resolve()}'")
        print("="*60)
else:
    print("A lista de bancos está vazia. Nenhuma análise foi executada.")

--- 3.1. Otimização: Pré-filtrando DataFrames para máxima performance... ---
 -> DataFrame IF.DATA pré-filtrado de 8,528,071 para 604,762 linhas.

--- 3.2. Coletando todos os componentes necessários... ---

 -> Coleta concluída. DataFrame original restaurado.

--- 3.3. Consolidando e exportando dados para o Power BI... ---

>>> SUCESSO! <<<
Relatório de COMPONENTES para o Power BI foi gerado com 4,494 linhas.
Arquivo salvo em: 'C:\Users\Enzo\Documents\Programming\bacen-data-analysis\Analysis\Results\relatorio_powerbi_202412.xlsx'
